# - Create your own Jupyter Kernel -

---
## Building your own Jupyter kernel is a three step process
1. Create/Pimp new virtual Python environment
   * venv
2. Create/Edit launch script for the Jupyter kernel
   * kernel.sh
3. Create/Edit Jupyter kernel configuration
   * kernel.json

### Settings

* Set new kernel name
  - change if you like

In [ ]:
export KERNEL_NAME=${USER}_kernel
echo ${KERNEL_NAME}

* Set directory for kernels virtual environment
  - change if you like

In [ ]:
export KERNEL_VENVS_DIR=${PROJECT}/${USER}/jupyter/kernels/
mkdir -p ${KERNEL_VENVS_DIR}
echo ${KERNEL_VENVS_DIR}

* Set location of kernel spec
  - select one:
    - personal kernel = "\${HOME}/.local/"  
    - project kernel  = "\${PROJECT}/.local/"  

In [ ]:
export KERNEL_SPECS_DIR=${HOME}/.local/
#export KERNEL_SPECS_DIR=${PROJECT}/.local/
echo ${KERNEL_SPECS_DIR}

---

## 1. Create/Pimp new virual Python environment

* 1.1 - Load required modules

In [ ]:
module -q purge
module -q use $OTHERSTAGES        
module -q load Stages/Devel-2019a 2> /dev/null
module -q load GCCcore/.8.3.0     2> /dev/null
module -q load Jupyter

* 1.2 - Load extra modules you need for your kernel

In [ ]:
# module load <module you need>

* 1.3 - Create and activate a virtual environment for the kernel  
and ensure python packages installed in the virtual environment are always prefered

In [ ]:
python -m venv --system-site-packages ${KERNEL_VENVS_DIR}/${KERNEL_NAME}
source ${KERNEL_VENVS_DIR}/${KERNEL_NAME}/bin/activate
export PYTHONPATH=${VIRTUAL_ENV}/lib/python3.6/site-packages:${PYTHONPATH}
echo ${VIRTUAL_ENV}

* 1.4 - Install Python libraries required for communication with Jupyter

In [ ]:
pip install --ignore-installed ipykernel

* 1.5 - Install whatever else you need in your Python virtual environment (using pip)

In [ ]:
#pip install <python-package you need>

---

## 2. Create/Edit launch script for the Jupyter kernel

* 2.1 - Create launch script, which loads your Python virtual environment and starts the ipykernel process inside:

In [ ]:
echo '#!/bin/bash

# Load required modules
module purge
module load $OTHERSTAGES'"
module load Stages/Devel-2019a
module load GCCcore/.8.3.0
module load Jupyter

# Load extra modules you need for your kernel (as you did in step 1.2)
#module load <module you need>
    
# Activate your Python virtual environment
source ${KERNEL_VENVS_DIR}/${KERNEL_NAME}/bin/activate
    
# Ensure python packages installed in the virtual environment are always prefered
export PYTHONPATH=${VIRTUAL_ENV}/lib/python3.6/site-packages:"'${PYTHONPATH}'"
    
exec python -m ipykernel "'$@' > ${VIRTUAL_ENV}/kernel.sh
chmod +x ${VIRTUAL_ENV}/kernel.sh

---

## 3. Create/Edit Jupyter kernel configuration

* 3.1 - Create Jupyter kernel configuration directory and files

In [ ]:
python -m ipykernel install --name=${KERNEL_NAME} --prefix ${KERNEL_SPECS_DIR}

* 3.2 - Adjust kernel.json file

In [ ]:
mv ${KERNEL_SPECS_DIR}/share/jupyter/kernels/${KERNEL_NAME}/kernel.json ${KERNEL_SPECS_DIR}/share/jupyter/kernels/${KERNEL_NAME}/kernel.json.orig

echo '
{
  "argv": [
    "'${KERNEL_VENVS_DIR}/${KERNEL_NAME}/kernel.sh'",
    "-m",
    "ipykernel_launcher",
    "-f",
    "{connection_file}"
  ],
  "display_name": "'${KERNEL_NAME}'",
  "language": "python"
}' > ${KERNEL_SPECS_DIR}/share/jupyter/kernels/${KERNEL_NAME}/kernel.json

---